In [3]:
import time
import pandas as pd
import numpy as np
from numpy import array
import torch
import torch.nn as nn
from sklearn.model_selection import KFold

In [4]:
save_path = "/Users/Brody1/Documents/Northwestern/Jiping/benchmarks/deep-learning/"
model_name = "fourier_one_features"
kf = KFold(n_splits = 10, shuffle =True)
num_epochs = 60

In [5]:
#### define functions ####

# Fully connected neural network with one hidden layer
class NeuralNet(nn.Module):
    def __init__(self, input_size, hidden_size, out_dim):
        super(NeuralNet, self).__init__()
        self.fc1 = nn.Linear(input_size, hidden_size)
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(hidden_size, out_dim)

    def forward(self, x):
        out = self.fc1(x)
        out = self.relu(out)
        out = self.fc2(out)
        return out


In [17]:
def display_fits(fits):
    print(f"Average correlation on tiling: {np.mean([fits[0][i] for i in range(fits[0].size) if (i % 4) == 2])}",
          f"\nAverage MSE on tiling: {np.mean([fits[1][i] for i in range(fits[1].size) if (i % 4) == 2])}",
          f"\nAverage correlation on random: {np.mean([fits[0][i] for i in range(fits[0].size) if (i % 4) == 1])}",
          f"\nAverage MSE on random: {np.mean([fits[1][i] for i in range(fits[1].size) if (i % 4) == 1])}",
          f"\nAverage correlation on ChrV: {np.mean([fits[0][i] for i in range(fits[0].size) if (i % 4) == 3])}",
          f"\nAverage MSE on ChrV: {np.mean([fits[1][i] for i in range(fits[1].size) if (i % 4) == 3])}",
          f"\nAverage correlation on CN: {np.mean([fits[0][i] for i in range(fits[0].size) if (i % 4) == 0])}",
          f"\nAverage MSE on CN: {np.mean([fits[1][i] for i in range(fits[1].size) if (i % 4) == 0])}")

In [6]:
# Construct sine and cosine matrices for Fourier features:
cos_matrix, sin_matrix = [[] for x in range(50)], [[] for x in range(50)]
for n in range(50):
    for k in range(50):
        cos_matrix[n].append(np.cos(2*np.pi*(n+1)*k/50))
        sin_matrix[n].append(np.sin(2*np.pi*(n+1)*k/50))

In [7]:
def dnaOneFourier(sequence):
    seq_array = array(list(sequence))
    code = {"A": [1], "C": [0], "G": [0], "T": [1],
            "a": [1], "c": [0], "g": [0], "t": [1]}
    AT_encoded_seq = []
    for char in seq_array:
        AT_encoded_seq.append(code[char][0])
    one_fourier_cos = np.matmul(array(AT_encoded_seq), array(cos_matrix).transpose())
    one_fourier_sin = np.matmul(array(AT_encoded_seq), array(sin_matrix).transpose())
    return list(np.concatenate((one_fourier_cos, one_fourier_sin)))

In [8]:
def find_c0new(dat):
  mat = np.empty((3,3), float)
  k = 2*np.pi/10.4
  n = array([26, 29, 31])
  mat[0:3,0] = 1
  mat[0:3, 1] = np.sin(n*k)
  mat[0:3, 2] = np.cos(n*k)
  inv_mat = np.linalg.inv(mat)
  c0A1A2 = array(np.matmul(dat[["n=26", "n=29", "n=31"]], inv_mat))
  c0Aphi = c0A1A2
  c0Aphi[:,0] = c0A1A2[:,0]
  c0Aphi[:,1] = np.sqrt(c0A1A2[:,1]**2 + c0A1A2[:,2]**2)
  c0Aphi[:,2] <- np.sign(c0A1A2[:,2]) * np.arccos(c0A1A2[:,1]/c0Aphi[:,1])
  return c0Aphi[:,0]

In [9]:
#### preparing data ####

data_cerevisiae_nucle = pd.read_csv("/Users/Brody1/Documents/Northwestern/Jiping/cycle1.txt",delimiter = ",")
X1 = []
for sequence_nt in data_cerevisiae_nucle["Sequence"]:
    X1.append(dnaOneFourier(sequence_nt))
X1 = np.float32(array(X1))
# X1 = X1.reshape((X1.shape[0],50,4,1))
### COMPUTE EIGENVECTORS:

# X1_reverse = np.flip(X1,[1,2])
# Y1 = data_cerevisiae_nucle["C0"].values.astype(float)
Y1 = np.float32(find_c0new(data_cerevisiae_nucle).astype(float))


In [10]:
data_random_library = pd.read_csv("/Users/Brody1/Documents/Northwestern/Jiping/cycle3.txt",delimiter = ",")
X3 = []
for sequence_nt in data_random_library["Sequence"]:
    X3.append(dnaOneFourier(sequence_nt))
X3 = np.float32(array(X3))
# X3 = X3.reshape((X3.shape[0],50,2,1))
# X3_reverse = np.flip(X3,[1,2])
# Y3 = data_random_library["C0"].values.astype(float)
Y3 = np.float32(find_c0new(data_random_library).astype(float))

In [11]:
data_tiling = pd.read_csv("/Users/Brody1/Documents/Northwestern/Jiping/cycle5.txt",delimiter = ",")
X5 = []
for sequence_nt in data_tiling["Sequence"]:
    X5.append(dnaOneFourier(sequence_nt))
X5 = np.float32(array(X5))
# X5 = X5.reshape((X5.shape[0],50,4,1))
# X5_reverse = np.flip(X5,[1,2])
# Y5 = data_tiling["C0"].values.astype(float)
Y5 = np.float32(find_c0new(data_tiling).astype(float))

In [12]:
data_chr5 = pd.read_csv("/Users/Brody1/Documents/Northwestern/Jiping/cycle6.txt",delimiter = ",")
X6 = []
for sequence_nt in data_chr5["Sequence"]:
    X6.append(dnaOneFourier(sequence_nt))
X6 = np.float32(array(X6))
# X6 = X6.reshape((X6.shape[0],50,4,1))
# X6_reverse = np.flip(X6,[1,2])
# Y6 = data_chr5["C0"].values.astype(float)
Y6 = np.float32(find_c0new(data_chr5).astype(float))

In [13]:
m1 = np.mean(Y1)
std1 = np.std(Y1)
Z1 = (Y1-m1)/std1

m3 = np.mean(Y3)
std3 = np.std(Y3)
Z3 = (Y3-m3)/std3


m5 = np.mean(Y5)
std5 = np.std(Y5)
Z5 = (Y5-m5)/std5


m6 = np.mean(Y6)
std6 = np.std(Y6)
Z6 = (Y6-m6)/std6

In [14]:
#### random

VALIDATION_LOSS = []
fold_var = 1
n = Z3.shape[0]
hidden_size = 50
learning_rate = 0.01

fits = []
detrend = []
times = []

for train_index, val_index in kf.split(Z3):
    training_X = np.float32(X3[train_index])
    validation_X = np.float32(X3[val_index])
    training_Y = np.float32(Z3[train_index])
    validation_Y = np.float32(Z3[val_index])

    # CREATE NEW MODEL
    model = NeuralNet(input_size=100, hidden_size=hidden_size, out_dim=1)
    # Loss and optimizer
    criterion = nn.MSELoss()
    optimizer = torch.optim.RMSprop(model.parameters(), lr=learning_rate)
    # CREATE CALLBACKS

    for epoch in range(num_epochs):
        inputs = torch.from_numpy(training_X)
        inputs = inputs.reshape(inputs.shape[0], 100)
        targets = torch.from_numpy(training_Y)
        targets = targets.reshape(targets.shape[0], 1)

        # Forward pass
        outputs = model(inputs)
        loss = criterion(outputs, targets)

        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if (epoch + 1) % 5 == 0:
            print('Epoch [{}/{}], Loss: {:.4f}'.format(epoch + 1, num_epochs, loss.item()))
    
    pred_Y = model(torch.from_numpy(training_X)).detach().numpy()
    pred_Y = pred_Y.reshape(pred_Y.shape[0])

    start_time = time.process_time()
    fit = model(torch.from_numpy(X1)).detach().numpy()
    fit = fit.reshape(fit.shape[0])
    fit_tmp =[np.corrcoef(fit, Z1)[0,1],np.mean(np.square(fit-Z1)),np.mean(fit),np.std(fit)]
    fits.append(fit_tmp)
    time0 = time.process_time() - start_time
    times.append([time0])
    
    start_time = time.process_time()
    fit = model(torch.from_numpy(validation_X)).detach().numpy()
    fit = fit.reshape(fit.shape[0])
    fit_tmp =[np.corrcoef(fit, validation_Y)[0,1],np.mean(np.square(fit-validation_Y)),np.mean(fit),np.std(fit)]
    fits.append(fit_tmp)
    time0 = time.process_time() - start_time
    times.append([time0])
    
    start_time = time.process_time()
    fit = model(torch.from_numpy(X5)).detach().numpy()
    fit = fit.reshape(fit.shape[0])
    fit_tmp =[np.corrcoef(fit, Z5)[0,1],np.mean(np.square(fit-Z5)),np.mean(fit),np.std(fit)]
    fits.append(fit_tmp)
    time0 = time.process_time() - start_time
    times.append([time0])
    
    start_time = time.process_time()
    fit = model(torch.from_numpy(X6)).detach().numpy()
    fit = fit.reshape(fit.shape[0])
    fit_tmp =[np.corrcoef(fit, Z6)[0,1],np.mean(np.square(fit-Z6)),np.mean(fit),np.std(fit)]
    fits.append(fit_tmp)
    time0 = time.process_time() - start_time
    times.append([time0])
    
    fold_var += 1

fits = array(fits)
fits = pd.DataFrame((fits))
fits.to_csv(save_path +model_name+"_fits_random.txt", index = False)

with open(save_path +model_name+"_time_random.txt", "w") as file:
    for row in times:
        s = " ".join(map(str, row))
        file.write(s+'\n')


Epoch [5/60], Loss: 1.1452
Epoch [10/60], Loss: 0.8494
Epoch [15/60], Loss: 0.7942
Epoch [20/60], Loss: 0.7587
Epoch [25/60], Loss: 0.7342
Epoch [30/60], Loss: 0.7169
Epoch [35/60], Loss: 0.7042
Epoch [40/60], Loss: 0.6942
Epoch [45/60], Loss: 0.6864
Epoch [50/60], Loss: 0.6798
Epoch [55/60], Loss: 0.6743
Epoch [60/60], Loss: 0.6695
Epoch [5/60], Loss: 0.9258
Epoch [10/60], Loss: 0.7520
Epoch [15/60], Loss: 0.7097
Epoch [20/60], Loss: 0.6840
Epoch [25/60], Loss: 0.6657
Epoch [30/60], Loss: 0.6516
Epoch [35/60], Loss: 0.6409
Epoch [40/60], Loss: 0.6322
Epoch [45/60], Loss: 0.6253
Epoch [50/60], Loss: 0.6201
Epoch [55/60], Loss: 0.6178
Epoch [60/60], Loss: 0.6205
Epoch [5/60], Loss: 1.1536
Epoch [10/60], Loss: 0.7891
Epoch [15/60], Loss: 0.7513
Epoch [20/60], Loss: 0.7251
Epoch [25/60], Loss: 0.7052
Epoch [30/60], Loss: 0.6896
Epoch [35/60], Loss: 0.6767
Epoch [40/60], Loss: 0.6654
Epoch [45/60], Loss: 0.6556
Epoch [50/60], Loss: 0.6470
Epoch [55/60], Loss: 0.6394
Epoch [60/60], Loss: 0.

In [15]:
print(fits)

           0         1         2         3
0   0.594261  0.724914  0.278317  0.618749
1   0.554297  0.725422  0.025980  0.624743
2   0.578904  0.665827 -0.004070  0.609573
3   0.461610  0.808932 -0.003879  0.609934
4   0.590966  0.725120  0.259714  0.674090
5   0.517871  0.749966  0.091781  0.638338
6   0.585328  0.659687 -0.021348  0.628224
7   0.471871  0.801541 -0.024770  0.625461
8   0.575896  0.860093  0.436456  0.611327
9   0.560209  0.703112  0.011554  0.602277
10  0.557572  0.721733  0.174249  0.605079
11  0.442919  0.860434  0.176573  0.602396
12  0.592064  0.740562  0.301027  0.614080
13  0.567323  0.689604  0.019057  0.580961
14  0.580643  0.663704  0.027739  0.589605
15  0.459491  0.806438  0.029691  0.588673
16  0.597280  0.822444  0.421268  0.638758
17  0.544669  0.701882 -0.018285  0.613975
18  0.578550  0.683828  0.130496  0.617530
19  0.460828  0.829230  0.134022  0.614549
20  0.592502  0.755624  0.326201  0.609083
21  0.573251  0.664405  0.005712  0.594345
22  0.58227

In [162]:
#### chrv

VALIDATION_LOSS = []
fold_var = 1
n = Z6.shape[0]
hidden_size = 50
learning_rate = 0.01

fits = []
detrend = []
times = []

for train_index, val_index in kf.split(Z6):
    training_X = np.float32(X6[train_index])
    validation_X = np.float32(X6[val_index])
    training_Y = np.float32(Z6[train_index])
    validation_Y = np.float32(Z6[val_index])

    # CREATE NEW MODEL
    model = NeuralNet(input_size=100, hidden_size=hidden_size, out_dim=1)
    # Loss and optimizer
    criterion = nn.MSELoss()
    optimizer = torch.optim.RMSprop(model.parameters(), lr=learning_rate)
    # CREATE CALLBACKS

    for epoch in range(num_epochs):
        inputs = torch.from_numpy(training_X)
        inputs = inputs.reshape(inputs.shape[0], 100)
        targets = torch.from_numpy(training_Y)
        targets = targets.reshape(targets.shape[0], 1)

        # Forward pass
        outputs = model(inputs)
        loss = criterion(outputs, targets)

        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if (epoch + 1) % 5 == 0:
            print('Epoch [{}/{}], Loss: {:.4f}'.format(epoch + 1, num_epochs, loss.item()))
    
    pred_Y = model(torch.from_numpy(training_X)).detach().numpy()
    pred_Y = pred_Y.reshape(pred_Y.shape[0])

    start_time = time.process_time()
    fit = model(torch.from_numpy(X1)).detach().numpy()
    fit = fit.reshape(fit.shape[0])
    fit_tmp =[np.corrcoef(fit, Z1)[0,1],np.mean(np.square(fit-Z1)),np.mean(fit),np.std(fit)]
    fits.append(fit_tmp)
    time0 = time.process_time() - start_time
    times.append([time0])
    
    start_time = time.process_time()
    fit = model(torch.from_numpy(X3)).detach().numpy()
    fit = fit.reshape(fit.shape[0])
    fit_tmp =[np.corrcoef(fit, Z3)[0,1],np.mean(np.square(fit-Z3)),np.mean(fit),np.std(fit)]
    fits.append(fit_tmp)
    time0 = time.process_time() - start_time
    times.append([time0])
    
    start_time = time.process_time()
    fit = model(torch.from_numpy(X5)).detach().numpy()
    fit = fit.reshape(fit.shape[0])
    fit_tmp =[np.corrcoef(fit, Z5)[0,1],np.mean(np.square(fit-Z5)),np.mean(fit),np.std(fit)]
    fits.append(fit_tmp)
    time0 = time.process_time() - start_time
    times.append([time0])
    
    start_time = time.process_time()
    fit = model(torch.from_numpy(validation_X)).detach().numpy()
    fit = fit.reshape(fit.shape[0])
    fit_tmp =[np.corrcoef(fit, validation_Y)[0,1],np.mean(np.square(fit-validation_Y)),np.mean(fit),np.std(fit)]
    fits.append(fit_tmp)
    time0 = time.process_time() - start_time
    times.append([time0])
    
    fold_var += 1

fits = array(fits)
fits = pd.DataFrame((fits))
fits.to_csv(save_path +model_name+"_fits_chrv.txt", index = False)

with open(save_path +model_name+"_time_chrv.txt", "w") as file:
    for row in times:
        s = " ".join(map(str, row))
        file.write(s+'\n')


Epoch [5/60], Loss: 1.8938
Epoch [10/60], Loss: 0.9418
Epoch [15/60], Loss: 0.8712
Epoch [20/60], Loss: 0.8402
Epoch [25/60], Loss: 0.8197
Epoch [30/60], Loss: 0.8055
Epoch [35/60], Loss: 0.7956
Epoch [40/60], Loss: 0.7886
Epoch [45/60], Loss: 0.7834
Epoch [50/60], Loss: 0.7797
Epoch [55/60], Loss: 0.7768
Epoch [60/60], Loss: 0.7744
Epoch [5/60], Loss: 1.3737
Epoch [10/60], Loss: 0.9851
Epoch [15/60], Loss: 0.9201
Epoch [20/60], Loss: 0.8821
Epoch [25/60], Loss: 0.8578
Epoch [30/60], Loss: 0.8414
Epoch [35/60], Loss: 0.8298
Epoch [40/60], Loss: 0.8212
Epoch [45/60], Loss: 0.8144
Epoch [50/60], Loss: 0.8089
Epoch [55/60], Loss: 0.8044
Epoch [60/60], Loss: 0.8006
Epoch [5/60], Loss: 1.6197
Epoch [10/60], Loss: 0.9571
Epoch [15/60], Loss: 0.8897
Epoch [20/60], Loss: 0.8555
Epoch [25/60], Loss: 0.8345
Epoch [30/60], Loss: 0.8200
Epoch [35/60], Loss: 0.8095
Epoch [40/60], Loss: 0.8016
Epoch [45/60], Loss: 0.7955
Epoch [50/60], Loss: 0.7905
Epoch [55/60], Loss: 0.7861
Epoch [60/60], Loss: 0.

In [163]:
#### CN

VALIDATION_LOSS = []
fold_var = 1
n = Z1.shape[0]
hidden_size = 50
learning_rate = 0.01

fits = []
detrend = []
times = []

for train_index, val_index in kf.split(Z1):
    training_X = np.float32(X1[train_index])
    validation_X = np.float32(X1[val_index])
    training_Y = np.float32(Z1[train_index])
    validation_Y = np.float32(Z1[val_index])

    # CREATE NEW MODEL
    model = NeuralNet(input_size=100, hidden_size=hidden_size, out_dim=1)
    # Loss and optimizer
    criterion = nn.MSELoss()
    optimizer = torch.optim.RMSprop(model.parameters(), lr=learning_rate)
    # CREATE CALLBACKS

    for epoch in range(num_epochs):
        inputs = torch.from_numpy(training_X)
        inputs = inputs.reshape(inputs.shape[0], 100)
        targets = torch.from_numpy(training_Y)
        targets = targets.reshape(targets.shape[0], 1)

        # Forward pass
        outputs = model(inputs)
        loss = criterion(outputs, targets)

        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if (epoch + 1) % 5 == 0:
            print('Epoch [{}/{}], Loss: {:.4f}'.format(epoch + 1, num_epochs, loss.item()))
    
    pred_Y = model(torch.from_numpy(training_X)).detach().numpy()
    pred_Y = pred_Y.reshape(pred_Y.shape[0])

    start_time = time.process_time()
    fit = model(torch.from_numpy(validation_X)).detach().numpy()
    fit = fit.reshape(fit.shape[0])
    fit_tmp =[np.corrcoef(fit, validation_Y)[0,1],np.mean(np.square(fit-validation_Y)),np.mean(fit),np.std(fit)]
    fits.append(fit_tmp)
    time0 = time.process_time() - start_time
    times.append([time0])
    
    start_time = time.process_time()
    fit = model(torch.from_numpy(X3)).detach().numpy()
    fit = fit.reshape(fit.shape[0])
    fit_tmp =[np.corrcoef(fit, Z3)[0,1],np.mean(np.square(fit-Z3)),np.mean(fit),np.std(fit)]
    fits.append(fit_tmp)
    time0 = time.process_time() - start_time
    times.append([time0])
    
    start_time = time.process_time()
    fit = model(torch.from_numpy(X5)).detach().numpy()
    fit = fit.reshape(fit.shape[0])
    fit_tmp =[np.corrcoef(fit, Z5)[0,1],np.mean(np.square(fit-Z5)),np.mean(fit),np.std(fit)]
    fits.append(fit_tmp)
    time0 = time.process_time() - start_time
    times.append([time0])
    
    start_time = time.process_time()
    fit = model(torch.from_numpy(X6)).detach().numpy()
    fit = fit.reshape(fit.shape[0])
    fit_tmp =[np.corrcoef(fit, Z6)[0,1],np.mean(np.square(fit-Z6)),np.mean(fit),np.std(fit)]
    fits.append(fit_tmp)
    time0 = time.process_time() - start_time
    times.append([time0])
    
    fold_var += 1

fits = array(fits)
fits = pd.DataFrame((fits))
fits.to_csv(save_path +model_name+"_fits_CN.txt", index = False)

with open(save_path +model_name+"_time_CN.txt", "w") as file:
    for row in times:
        s = " ".join(map(str, row))
        file.write(s+'\n')


Epoch [5/60], Loss: 2.3442
Epoch [10/60], Loss: 0.9713
Epoch [15/60], Loss: 0.8410
Epoch [20/60], Loss: 0.7805
Epoch [25/60], Loss: 0.7432
Epoch [30/60], Loss: 0.7181
Epoch [35/60], Loss: 0.6999
Epoch [40/60], Loss: 0.6863
Epoch [45/60], Loss: 0.6754
Epoch [50/60], Loss: 0.6665
Epoch [55/60], Loss: 0.6588
Epoch [60/60], Loss: 0.6519
Epoch [5/60], Loss: 1.3257
Epoch [10/60], Loss: 0.7826
Epoch [15/60], Loss: 0.7196
Epoch [20/60], Loss: 0.6998
Epoch [25/60], Loss: 0.6885
Epoch [30/60], Loss: 0.6798
Epoch [35/60], Loss: 0.6723
Epoch [40/60], Loss: 0.6656
Epoch [45/60], Loss: 0.6594
Epoch [50/60], Loss: 0.6536
Epoch [55/60], Loss: 0.6505
Epoch [60/60], Loss: 0.6701
Epoch [5/60], Loss: 1.4790
Epoch [10/60], Loss: 0.9680
Epoch [15/60], Loss: 0.8651
Epoch [20/60], Loss: 0.8109
Epoch [25/60], Loss: 0.7749
Epoch [30/60], Loss: 0.7486
Epoch [35/60], Loss: 0.7282
Epoch [40/60], Loss: 0.7120
Epoch [45/60], Loss: 0.6985
Epoch [50/60], Loss: 0.6877
Epoch [55/60], Loss: 0.6781
Epoch [60/60], Loss: 0.

In [164]:
print(fits)

           0         1         2         3
0   0.580772  0.659283 -0.002047  0.631271
1   0.499352  0.857777 -0.249705  0.710957
2   0.548732  0.813266 -0.299729  0.705372
3   0.432755  0.977771 -0.303229  0.703125
4   0.577450  0.713217 -0.199187  0.626237
5   0.495847  1.078646 -0.515294  0.738712
6   0.540857  0.982943 -0.494656  0.716314
7   0.428603  1.142328 -0.495226  0.712820
8   0.537355  0.706825  0.001726  0.617399
9   0.481694  0.892503 -0.272453  0.705975
10  0.529213  0.825892 -0.284226  0.687878
11  0.420780  0.976534 -0.286725  0.687946
12  0.546918  0.839280  0.338472  0.542846
13  0.499460  0.760358  0.028604  0.594333
14  0.537224  0.726268  0.104909  0.599446
15  0.422351  0.864306  0.105174  0.600185
16  0.490513  0.782137  0.016641  0.655864
17  0.460435  0.939219 -0.295250  0.713510
18  0.498921  0.865613 -0.260987  0.714376
19  0.394683  1.012208 -0.258131  0.713041
20  0.584588  0.630891  0.145385  0.590723
21  0.504818  0.800184 -0.163465  0.673057
22  0.54948

In [16]:
#### tiling

VALIDATION_LOSS = []
fold_var = 1
n = Y5.shape[0]
hidden_size = 50
learning_rate = 0.01

fits = []
detrend = []
times = []

for train_index, val_index in kf.split(Y5):
    training_X = np.float32(X5[train_index])
    validation_X = np.float32(X5[val_index])
    training_Y = np.float32(Y5[train_index])
    validation_Y = np.float32(Y5[val_index])

    # CREATE NEW MODEL
    model = NeuralNet(input_size=100, hidden_size=hidden_size, out_dim=1)
    # Loss and optimizer
    criterion = nn.MSELoss()
    optimizer = torch.optim.RMSprop(model.parameters(), lr=learning_rate)
    # CREATE CALLBACKS

    for epoch in range(num_epochs):
        inputs = torch.from_numpy(training_X)
        inputs = inputs.reshape(inputs.shape[0], 100)
        targets = torch.from_numpy(training_Y)
        targets = targets.reshape(targets.shape[0], 1)

        # Forward pass
        outputs = model(inputs)
        loss = criterion(outputs, targets)

        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if (epoch + 1) % 5 == 0:
            print('Epoch [{}/{}], Loss: {:.4f}'.format(epoch + 1, num_epochs, loss.item()))
    
    pred_Y = model(torch.from_numpy(training_X)).detach().numpy()
    pred_Y = pred_Y.reshape(pred_Y.shape[0])

    start_time = time.process_time()
    fit = model(torch.from_numpy(X1)).detach().numpy()
    fit = fit.reshape(fit.shape[0])
    fit_tmp =[np.corrcoef(fit, Y1)[0,1],np.mean(np.square(fit-Y1)),np.mean(fit),np.std(fit)]
    fits.append(fit_tmp)
    time0 = time.process_time() - start_time
    times.append([time0])
    
    start_time = time.process_time()
    fit = model(torch.from_numpy(X3)).detach().numpy()
    fit = fit.reshape(fit.shape[0])
    fit_tmp =[np.corrcoef(fit, Y3)[0,1],np.mean(np.square(fit-Y3)),np.mean(fit),np.std(fit)]
    fits.append(fit_tmp)
    time0 = time.process_time() - start_time
    times.append([time0])
    
    start_time = time.process_time()
    fit = model(torch.from_numpy(validation_X)).detach().numpy()
    fit = fit.reshape(fit.shape[0])
    fit_tmp =[np.corrcoef(fit, validation_Y)[0,1],np.mean(np.square(fit-validation_Y)),np.mean(fit),np.std(fit)]
    fits.append(fit_tmp)
    time0 = time.process_time() - start_time
    times.append([time0])
    
    start_time = time.process_time()
    fit = model(torch.from_numpy(X6)).detach().numpy()
    fit = fit.reshape(fit.shape[0])
    fit_tmp =[np.corrcoef(fit, Y6)[0,1],np.mean(np.square(fit-Y6)),np.mean(fit),np.std(fit)]
    fits.append(fit_tmp)
    time0 = time.process_time() - start_time
    times.append([time0])
    
    fold_var += 1

fits = array(fits)
fits = pd.DataFrame((fits))
fits.to_csv(save_path +model_name+"_fits_tiling.txt", index = False)

with open(save_path +model_name+"_time_tiling.txt", "w") as file:
    for row in times:
        s = " ".join(map(str, row))
        file.write(s+'\n')


Epoch [5/60], Loss: 0.7920
Epoch [10/60], Loss: 0.2726
Epoch [15/60], Loss: 0.2092
Epoch [20/60], Loss: 0.1842
Epoch [25/60], Loss: 0.1699
Epoch [30/60], Loss: 0.1602
Epoch [35/60], Loss: 0.1528
Epoch [40/60], Loss: 0.1470
Epoch [45/60], Loss: 0.1422
Epoch [50/60], Loss: 0.1381
Epoch [55/60], Loss: 0.1346
Epoch [60/60], Loss: 0.1316
Epoch [5/60], Loss: 0.6497
Epoch [10/60], Loss: 0.2890
Epoch [15/60], Loss: 0.2419
Epoch [20/60], Loss: 0.2123
Epoch [25/60], Loss: 0.1916
Epoch [30/60], Loss: 0.1762
Epoch [35/60], Loss: 0.1643
Epoch [40/60], Loss: 0.1549
Epoch [45/60], Loss: 0.1472
Epoch [50/60], Loss: 0.1409
Epoch [55/60], Loss: 0.1356
Epoch [60/60], Loss: 0.1312
Epoch [5/60], Loss: 0.4841
Epoch [10/60], Loss: 0.2370
Epoch [15/60], Loss: 0.1969
Epoch [20/60], Loss: 0.1731
Epoch [25/60], Loss: 0.1572
Epoch [30/60], Loss: 0.1460
Epoch [35/60], Loss: 0.1380
Epoch [40/60], Loss: 0.1320
Epoch [45/60], Loss: 0.1276
Epoch [50/60], Loss: 0.1242
Epoch [55/60], Loss: 0.1215
Epoch [60/60], Loss: 0.

In [18]:
display_fits(fits)

Average correlation on tiling: 0.5119961597181356 
Average MSE on tiling: 0.12354688867926597 
Average correlation on random: 0.45938177087521676 
Average MSE on random: 0.11350463777780533 
Average correlation on ChrV: 0.4041449483872988 
Average MSE on ChrV: 0.22059142738580703 
Average correlation on CN: 0.5235572529772334 
Average MSE on CN: 0.16049545854330063
